In [1]:
from lbpqca.lattice.reductions_lattice import LLL, BKZ, babai_nearest_plane
from lbpqca.lattice.attacks import BDD_atack, BDD_atack_recovery_secret, SVP_atack, SVP_atack_recovery_secret
import numpy as np

In [2]:
A=np.array([[ 4, 13, 11,  0,  6],
 [14,  9,  0, 1, 12],
 [14,  2,  1, 14,  0],
 [ 9,  1,  5,  8,  7],
 [ 6,  0,  0,  2,  0],
 [11,  8, 11,  15, 10],
 [12,  6,  7, 16, 13],
 [16,  6, 11, 16, 11],
 [14, 11, 11,  6, 14],
 [ 2,  9, 12, 14,  8]],dtype=int)

m, n, q = 10, 5, 17

e=np.array([ 0, 0,  1,  0,  0, 0, -1, -1, 0, 1],dtype=int)

s=np.array([2,2,4,5,1],dtype=int)

b=A@s+e

print('A=',A,'\nb=',b)



A= [[ 4 13 11  0  6]
 [14  9  0  1 12]
 [14  2  1 14  0]
 [ 9  1  5  8  7]
 [ 6  0  0  2  0]
 [11  8 11 15 10]
 [12  6  7 16 13]
 [16  6 11 16 11]
 [14 11 11  6 14]
 [ 2  9 12 14  8]] 
b= [ 84  63 107  87  22 167 156 178 138 149]


Primary attack is achieved by constructing a lattice from our LWE pair $(A, \mathbf{b})$ thru some embedding method and then by using lattice reduction algorithm we can obtain a short vector that should allow us to reconstruct the secret vector $\mathbf{s}$.

First we will reduce the LWE problem to BDD by constructing the associated q-ary lattice
$$
    \mathcal{L'} = \left\{ \mathbf{y'} \in \mathbb{Z}^m : \mathbf{y'} = A\mathbf{x'}\mod q, \quad \forall \mathbf{x'} \in \mathbb{Z}^n \right \}
$$
We can try to solve the BDD directly by using babai's nearest plane algorithm.

In [3]:
print("LWE as BDD problem:")
BDD_Basis =  BDD_atack(A, q).astype(int)
print(BDD_Basis)

LWE as BDD problem:
[[ 1  0  0  0 14  0  9  0 11 14]
 [ 0  1  0  0  7  0 15  2 16  2]
 [ 0  0  1  0  0  0  9  0 15  7]
 [ 0  0  0  1 10  0 16  7 13  5]
 [ 0  0  0  0 17  0  0  0  0  0]
 [ 0  0  0  0  0  1 16  4  9 12]
 [ 0  0  0  0  0  0 17  0  0  0]
 [ 0  0  0  0  0  0  0 17  0  0]
 [ 0  0  0  0  0  0  0  0 17  0]
 [ 0  0  0  0  0  0  0  0  0 17]]


In [4]:
w = babai_nearest_plane(BDD_Basis, b).astype(int)
Aw = np.block([A, w.reshape(-1,1)])
sw=BDD_atack_recovery_secret(A,w,q)
print("recovered secret:")
print(sw)
print("true secret:")
print(s)

recovered secret:
[2 2 4 5 1]
true secret:
[2 2 4 5 1]


Now we can use Kannan's embedding.
For Kannan's embedding we define the embedding lattice $\mathcal{L_K}$ as follows
$$
\mathcal{L_K} := \left\{ \mathbf{y} \in \mathbb{Z}^{m + 1} : \mathbf{y} = \bar{A}x \mod q, \quad \forall \mathbf{x} \in \mathbb{Z}^{n+1}, \bar{A} =  \begin{pmatrix} A & \mathbf{b} \newline \mathbf{0} & 1 \end{pmatrix}
\right \}
$$
If the columns of the matrix $\bar{a}$ are linearly independed (given the nature of LWE there is a very high chance they are), then $\mathbf{v}^T = (\mathbf{e}^T| 1)$ is a short vector in the lattice. We can find it by using some lattice reduction algorithm e.g. BKZ.

In [5]:
h=SVP_atack(A,b,q)
print("Kannan's embedding")
print(h)

Kannan's embedding
[[ 1  0  0  0 14  0  0 13  7 14  6]
 [ 0  1  0  0  7  0  0  9  2 10 10]
 [ 0  0  1  0  0  0  0 11 10  5  6]
 [ 0  0  0  1 10  0  0  2  6  9  5]
 [ 0  0  0  0 17  0  0  0  0  0  0]
 [ 0  0  0  0  0  1  0 16  2 16  5]
 [ 0  0  0  0  0  0  1 12 10  4  5]
 [ 0  0  0  0  0  0  0 17  0  0  0]
 [ 0  0  0  0  0  0  0  0 17  0  0]
 [ 0  0  0  0  0  0  0  0  0 17  0]
 [ 0  0  0  0  0  0  0  0  0  0 17]]


In [6]:
x=BKZ(h,10)
print('Reduces basis')
print(x)

Reduces basis
[[ 0  0 -1  0  0  0  1  1  0 -1 -1]
 [-1  0 -1 -2  0  0  1  1 -2  1  0]
 [-1  1  0 -1  0  1 -1 -2 -2 -1 -1]
 [-2  0 -2  1 -1 -1 -2 -1  1 -2  0]
 [-2 -2  1 -1 -1 -1  0  0  1  0 -2]
 [-1 -1  2 -3  0  2  2 -1  0 -1  1]
 [-1  3  0  1  0  2 -1  2 -1  2  0]
 [ 2  2 -1 -1 -2 -1  0 -2  0  1 -1]
 [ 0 -2 -1  0  3  0 -2 -2  0  1 -2]
 [-3  0 -1  1  2 -1  1 -1  0  1 -2]
 [-1 -3 -2  0 -1  2  1 -1 -2 -1  1]]


In [7]:
sw1=SVP_atack_recovery_secret(A,b,x[0],q)
print("recovered secret:")
print(sw1)
print("true secret:")
print(s)

recovered secret:
[2 2 4 5 1]
true secret:
[2 2 4 5 1]
